# Regularizadores 

📌 Regularización, es una herramienta para disminuir la complejidad de los modelos, por medio de una penalización a las variables más irrelevantes (aportan menor información).

La **regularización** es una píldora contra el overfitting; su funcionamiento consiste en añadir un bias  (sesgo) a los datos cuando penaliza a las variables, y a su vez le quita varianza a los datos.

Este cambio ayuda al modelo a generalizar mejor.

> 💡 **Regularizer** → +Sesgo por -Varianza. <br>
**Varianza** → Distancia entre los verdes (valores reales) hacia la línea de regresión (predicción)

![Ejemplo de overfitting](../imgs/overfitting.png)

**¿Donde se realiza la penalización de las variables?**
La penalización se aplica en la función de perdida, también llamada función de costo. 
La función de perdida, mide que tan bien aprendió el modelo, mediante la medición de la distancia de los valores reales a la linea de regresión (modelo/predicción).
En modelos de regresión, a la función de perdida se le conoce como *Error Cuadrático Medio*, ecuación es la siguiente:

$$
ECM = {\frac 1 N} {\sum_{i=1}^N}(y_i-y_{i,pred)^2}
$$

> 💡 Menor pedida = Mejor rendimiento del modelo.

**¿Cómo se aplica la regularización?**

Existen tres tipos de regularizadores, cada uno actúa de una forma especifica sobre la función de perdida, la selección de uno u otro dependerá del objetivo del modelo.

**Tipos de regularizadores:**
- **L1 (Lasso)**: Elimina features que no aporten demasiado valor al modelo (los convierte en 0). Probar Lasso cuando hay pocos features relacionados a la variable a predecir.
- **L2 (Ridge)**: Reduce el impacto de feautres que no aporten demasiado valor al modelo (nunca son 0). Probar Ridge si hay varios features relacionado con la variable a predecir.
- **ElasticNet**: Combinación de Lasso y Ridge.

### Ejemplo de utilizar regularizadores

Para este ejemplo se utilizará el Dataset _World Happiness_ de año 2017, con el cuál se busca predecir el `score` que mide la felicidad de las personas en diferentes regiones del mundo dados ciertos parámetros demográficos, económicos, politicos, etc.





<div class="alert alert-info", role="alert">
    <h5>⚠️</h5>
    <p>
      El ejemplo desglosado a continuación sigue paso a paso la implementación que se realizó en el curso, solo con unos detalles adicionales. <br>
			Especificamente en la clase <a href="https://platzi.com/cursos/scikitlearn/implementacion-de-lasso-y-ridge/"> 
				Implementación de Regularización en Modelos de Regresión Lineal </a>
				<br>
				Al ser un ejemplo didactico se salta varios pasos fundamentales de un proyecto de Data Science.
    </p>
</div>

#### Librerias

In [1]:
# Manipulación y visualización de datos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos de regresión
from sklearn.linear_model import LinearRegression # Regresión lineal
from sklearn.linear_model import Lasso # Regresión Laso 
from sklearn.linear_model import Ridge

# Divir dataset en entrenamiento y pruebas
from sklearn.model_selection import train_test_split

# Fución de perdida -> Calcular el error del modelo.
from sklearn.metrics import mean_absolute_error 

In [2]:
%run 0.0-ml_professional-setup.ipynb

In [3]:
file_path = path.data_raw_dir("2017.csv")
df = pd.read_csv(file_path)

df.columns = ["country","rank","score","high","low","gdp","family","lifexp","freedom","generosity","corruption","dystopia"]

display(df.head(), df.describe().T)

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


,count,mean,std,min,25%,50%,75%,max
rank,155.0,78.000000,44.888751,1.000000,39.500000,78.000000,116.500000,155.000000
score,155.0,5.354019,1.131230,2.693000,4.505500,5.279000,6.101500,7.537000
high,155.0,5.452326,1.118542,2.864884,4.608172,5.370032,6.194600,7.622030
low,155.0,5.255713,1.145030,2.521116,4.374955,5.193152,6.006527,7.479556
gdp,155.0,0.984718,0.420793,0.000000,0.663371,1.064578,1.318027,1.870766
family,155.0,1.188898,0.287263,0.000000,1.042635,1.253918,1.414316,1.610574
lifexp,155.0,0.551341,0.237073,0.000000,0.369866,0.606042,0.723008,0.949492
freedom,155.0,0.408786,0.149997,0.000000,0.303677,0.437454,0.516561,0.658249
generosity,155.0,0.246883,0.134780,0.000000,0.154106,0.231538,0.323762,0.838075
corruption,155.0,0.123120,0.101661,0.000000,0.057271,0.089848,0.153296,0.464308


Separamos el dataset en:
- Features de entrenamiento: 
	- `gdp`, `family`, `lifexp`, `freedom`, `corruption`, `generosity`, `dystopia`
- Target: `score`

In [4]:
X = df[['gdp', 'family', 'lifexp', 'freedom' , 'corruption' , 'generosity', 'dystopia']]
y = df[["score"]]

Dividimos los datos en entrenamiento y pruebas:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

Entrenamiento de los modelos de regresión:

In [6]:
# Regresión Lineal
lr_model = LinearRegression().fit(X_train, y_train)
y_predict_lr = lr_model.predict(X_test)

# Lasso Regresion
lasso_model = Lasso(alpha=.02).fit(X_train, y_train)
y_predict_lasso = lasso_model.predict(X_test)

# Ridge Regresion
ridge_model = Ridge(alpha=1).fit(X_train, y_train)
y_predict_ridge = ridge_model.predict(X_test)

📌**NOTA**:
- Lasso busca forzar los coeficientes a ser `0` eliminando características, al enviar el valor `alpha = 0.02` en el modelo Lasso permite más flexibilidad para dicho ajuste.
- Por otro lado Ridge busca reducir el impacto de las variables, por lo tanto un `alpha = 1` en el modelo penaliza menos la magnitud de los coeficientes.

Calculo y comparación de la función de perdida en cada uno de los modelos:

In [7]:
mse_linear = mean_absolute_error(y_test, y_predict_lr)
mse_lasso = mean_absolute_error(y_test, y_predict_lasso)
mse_ridge = mean_absolute_error(y_test, y_predict_ridge)

print(f"Linear Loss: {mse_linear}")
print(f"Lasso Loss: {mse_lasso}")
print(f"Ridge Loss: {mse_ridge}")

Linear Loss: 0.0002783763061961586
Lasso Loss: 0.18305279945371006
Ridge Loss: 0.062347840067435024


La función de perdida, _Error Cuadratico Medio_ (MSE por sus siglas en ingés), es el promedio de las distancias de los puntos reales contra los predichos al cuadrado, entre más pequeño sea este número mejores resultados obtenemos del modelo.

Segun MSE de los tres modelos entrenados, el que obtuvo mejores resultados fue el modelo de _Regresión Lineal_, para este ejemplo, utilizar alguna de las penalizaciones ya sea Lasso (L1) o Ridge (L2) no es necesario, pues sus resultados no son tan buenos como la regresión lineal tradicional.


Coeficientes de cada feature:

In [8]:
coef_df = pd.DataFrame([lr_model.coef_[0], lasso_model.coef_, ridge_model.coef_],
                       columns= X.columns,
                       index=["Linear Regresion", "Lasso", "Ridge"])

coef_df

,gdp,family,lifexp,freedom,corruption,generosity,dystopia
Linear Regresion,1.000128,0.999946,0.999835,1.000034,0.999771,1.000260,0.999938
Lasso,1.289214,0.919694,0.476864,0.732973,0.000000,0.142455,0.899653
Ridge,1.072349,0.970486,0.856054,0.874002,0.685833,0.732857,0.962066
